# Tuning des modèles en enlevant Hillshade_9am

In [2]:
import pandas as pd
import scipy.stats as stats
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

data = pd.read_csv('train.csv')

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# On sélectionne les colonnes d'entrée (features) et la cible
X = data.drop(columns=['Cover_Type'])
y = data['Cover_Type']

# On sélectionne les colonnes d'entrée (features) et la cible
X = data.drop(columns=['Cover_Type'])
y = data['Cover_Type']

# On divise les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# On normalise les données continues 
from sklearn.preprocessing import StandardScaler

continuous_columns = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 
                      'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 
                      'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points']
scaler = StandardScaler()

# On applique le scaler uniquement sur les colonnes continues
#X_train[continuous_columns] = scaler.fit_transform(X_train[continuous_columns])
#X_test[continuous_columns] = scaler.transform(X_test[continuous_columns])

X_train_hill = X_train.drop(columns=['Hillshade_9am'])
X_test_hill = X_test.drop(columns=['Hillshade_9am'])

In [4]:
from sklearn.model_selection import cross_val_score
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

## Random Forest

In [5]:
from sklearn.ensemble import RandomForestClassifier

# On définit l'espace de recherche pour Random Forest
space_rf = {
    'n_estimators': hp.choice('n_estimators', [100, 200, 300, 400, 500]),
    'max_depth': hp.choice('max_depth', [5, 10, 15, 20, None]),
    'min_samples_split': hp.choice('min_samples_split', [2, 5, 10]),
    'min_samples_leaf': hp.choice('min_samples_leaf', [1, 2, 4]),
    'max_features': hp.choice('max_features', ['sqrt', 'log2'])  # Correction : 'auto' remplacé par 'sqrt'
}

# On crée la fonction objectif pour Random Forest
def objective_rf(params):
    # On initialise le modèle avec les paramètres proposés
    model = RandomForestClassifier(**params, random_state=42)
    # On effectue une validation croisée pour évaluer la performance
    score = cross_val_score(model, X_train_hill, y_train, cv=7, scoring='accuracy').mean()
    # On retourne la perte (score négatif) car Hyperopt minimise cette fonction
    return {'loss': -score, 'status': STATUS_OK}

# On lance Hyperopt pour optimiser les hyperparamètres de Random Forest
trials_rf = Trials()
best_params_rf = fmin(fn=objective_rf, space=space_rf, algo=tpe.suggest, max_evals=50, trials=trials_rf)
print("Meilleurs paramètres Random Forest :", best_params_rf)

100%|██████████| 50/50 [14:28<00:00, 17.37s/trial, best loss: -0.8778344671201813]
Meilleurs paramètres Random Forest : {'max_depth': 4, 'max_features': 0, 'min_samples_leaf': 0, 'min_samples_split': 0, 'n_estimators': 2}


In [6]:
# Traduction des indices en valeurs concrètes
best_rf_params = {
    'max_depth': [5, 10, 15, 20, None][4],           
    'max_features': ['sqrt', 'log2'][0],             
    'min_samples_leaf': [1, 2, 4][0],                
    'min_samples_split': [2, 5, 10][0],              
    'n_estimators': [100, 200, 300, 400, 500][2]     
}

# Initialiser le modèle avec les valeurs concrètes
rf_model = RandomForestClassifier(**best_rf_params, random_state=42)

# Entraîner le modèle sur l'ensemble d'entraînement
rf_model.fit(X_train_hill, y_train)

# Prédire sur l'ensemble de test
y_pred_rf = rf_model.predict(X_test_hill)

# Évaluer les performances du modèle
from sklearn.metrics import accuracy_score, classification_report
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Précision Random Forest :", accuracy_rf)
print("Rapport de classification Random Forest :\n", classification_report(y_test, y_pred_rf))

Précision Random Forest : 0.8847001763668431
Rapport de classification Random Forest :
               precision    recall  f1-score   support

           1       0.81      0.79      0.80       648
           2       0.82      0.73      0.77       648
           3       0.86      0.90      0.88       648
           4       0.95      0.98      0.96       648
           5       0.92      0.96      0.94       648
           6       0.89      0.88      0.89       648
           7       0.93      0.95      0.94       648

    accuracy                           0.88      4536
   macro avg       0.88      0.88      0.88      4536
weighted avg       0.88      0.88      0.88      4536



## XGBoost

In [15]:
import xgboost as xgb

y_train_adj = y_train - 1
y_test_adj = y_test - 1

In [14]:
# On définit l'espace de recherche pour XGBoost
space_xgb = {
    'n_estimators': hp.choice('n_estimators', [100, 200, 300, 400, 500]),
    'max_depth': hp.choice('max_depth', [3, 6, 9, 12]),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
    'gamma': hp.uniform('gamma', 0, 0.5)
}

# On crée la fonction objectif pour XGBoost
def objective_xgb(params):
    # On initialise XGBoost avec les paramètres proposés
    model = xgb.XGBClassifier(objective='multi:softmax', num_class=7, **params, random_state=42)
    # On utilise une validation croisée pour calculer la précision
    score = cross_val_score(model, X_train_hill, y_train_adj, cv=7, scoring='accuracy').mean()
    # On retourne la perte (score négatif) car Hyperopt minimise cette fonction
    return {'loss': -score, 'status': STATUS_OK}

# On lance l'optimisation avec Hyperopt pour XGBoost
trials_xgb = Trials()
best_params_xgb = fmin(fn=objective_xgb, space=space_xgb, algo=tpe.suggest, max_evals=50, trials=trials_xgb)
print("Meilleurs paramètres XGBoost :", best_params_xgb)

100%|██████████| 50/50 [12:22<00:00, 14.86s/trial, best loss: -0.8839758125472411]
Meilleurs paramètres XGBoost : {'colsample_bytree': 0.9935889419671114, 'gamma': 0.012860005064671678, 'learning_rate': 0.11209385123371478, 'max_depth': 2, 'n_estimators': 1, 'subsample': 0.7078887451962781}


In [15]:
import xgboost as xgb

# On utilise les meilleurs hyperparamètres trouvés par Hyperopt
best_xgb_params = {
    'colsample_bytree': 0.9935889419671114,
    'gamma': 0.012860005064671678,
    'learning_rate': 0.11209385123371478,
    'max_depth': 9,
    'n_estimators': [100, 200, 300, 400, 500][1],  
    'subsample': 0.7078887451962781
}

# Initialiser le modèle XGBoost avec les paramètres optimaux
xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=7, **best_xgb_params, random_state=42)

# Entraîner le modèle sur l'ensemble d'entraînement
xgb_model.fit(X_train_hill, y_train_adj)

# Prédire les classes sur l'ensemble de test
y_pred_xgb = xgb_model.predict(X_test_hill)

# Évaluer les performances du modèle
accuracy_xgb = accuracy_score(y_test_adj, y_pred_xgb)
print("Précision du modèle XGBoost :", accuracy_xgb)
print("Rapport de classification XGBoost :\n", classification_report(y_test_adj, y_pred_xgb))

Précision du modèle XGBoost : 0.88668430335097
Rapport de classification XGBoost :
               precision    recall  f1-score   support

           0       0.80      0.79      0.79       648
           1       0.79      0.72      0.76       648
           2       0.89      0.90      0.89       648
           3       0.95      0.98      0.97       648
           4       0.92      0.95      0.93       648
           5       0.90      0.91      0.91       648
           6       0.94      0.96      0.95       648

    accuracy                           0.89      4536
   macro avg       0.88      0.89      0.89      4536
weighted avg       0.88      0.89      0.89      4536



## LightGBM

In [16]:
import lightgbm as lgb

# On définit l'espace de recherche pour LightGBM
space_lgb = {
    'n_estimators': hp.choice('n_estimators', [100, 200, 300, 400, 500]),
    'max_depth': hp.choice('max_depth', [5, 10, 15, 20, -1]),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
    'num_leaves': hp.choice('num_leaves', [31, 50, 70, 100]),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0)
}

# On crée la fonction objectif pour LightGBM
def objective_lgb(params):
    # On initialise LightGBM avec les paramètres proposés
    model = lgb.LGBMClassifier(objective='multiclass', **params, verbosity = -1,random_state=42)
    # On utilise la validation croisée pour évaluer la précision
    score = cross_val_score(model, X_train_hill, y_train, cv=7, scoring='accuracy').mean()
    # On retourne la perte (score négatif) car Hyperopt minimise cette fonction
    return {'loss': -score, 'status': STATUS_OK}

# On lance l'optimisation avec Hyperopt pour LightGBM
trials_lgb = Trials()
best_params_lgb = fmin(fn=objective_lgb, space=space_lgb, algo=tpe.suggest, max_evals=50, trials=trials_lgb)
print("Meilleurs paramètres LightGBM :", best_params_lgb)

100%|██████████| 50/50 [49:50<00:00, 59.81s/trial, best loss: -0.8915343915343915]   
Meilleurs paramètres LightGBM : {'colsample_bytree': 0.6284626649933062, 'learning_rate': 0.08427513495693706, 'max_depth': 2, 'n_estimators': 1, 'num_leaves': 3, 'subsample': 0.6894988308344379}


In [20]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score, classification_report

# Meilleurs paramètres de LightGBM (après avoir traduit les indices en valeurs)
best_lgb_params = {
    'colsample_bytree': 0.6284626649933062,
    'learning_rate': 0.08427513495693706,
    'max_depth': 15,
    'n_estimators': [100, 200, 300, 400, 500][1],  
    'num_leaves': [31, 50, 70, 100][3],             
    'subsample': 0.6894988308344379,
    'verbosity': -1  # Désactive les avertissements pendant l'entraînement
}

# Initialiser le modèle LightGBM avec les paramètres optimisés
lgb_model = lgb.LGBMClassifier(objective='multiclass', num_class=7, **best_lgb_params, random_state=42)

# Entraîner le modèle sur l'ensemble d'entraînement
lgb_model.fit(X_train_hill, y_train)

# Prédire les classes sur l'ensemble de test
y_pred_lgb = lgb_model.predict(X_test_hill)

# Évaluer les performances du modèle
accuracy_lgb = accuracy_score(y_test, y_pred_lgb)
print("Précision du modèle LightGBM :", accuracy_lgb)
print("Rapport de classification LightGBM :\n", classification_report(y_test, y_pred_lgb))

Précision du modèle LightGBM : 0.8941798941798942
Rapport de classification LightGBM :
               precision    recall  f1-score   support

           1       0.82      0.80      0.81       648
           2       0.81      0.75      0.78       648
           3       0.89      0.90      0.89       648
           4       0.96      0.98      0.97       648
           5       0.93      0.96      0.94       648
           6       0.90      0.92      0.91       648
           7       0.94      0.95      0.95       648

    accuracy                           0.89      4536
   macro avg       0.89      0.89      0.89      4536
weighted avg       0.89      0.89      0.89      4536



## Extra Trees

In [17]:
from sklearn.ensemble import ExtraTreesClassifier

# Définir l'espace de recherche pour ExtraTreesClassifier
space_et = {
    'n_estimators': hp.choice('n_estimators', [100, 200, 300, 400, 500]),  # Choix pour le nombre d'arbres
    'max_depth': hp.choice('max_depth', [None, 10, 20, 30, 40]),           # Profondeur maximale
    'min_samples_split': hp.choice('min_samples_split', [2, 5, 10]),       # Nombre minimum d'échantillons pour diviser
    'min_samples_leaf': hp.choice('min_samples_leaf', [1, 2, 4]),          # Nombre minimum d'échantillons dans une feuille
    'max_features': hp.choice('max_features', ['sqrt', 'log2', None])      # Nombre de caractéristiques pour chaque division
}

# Fonction objectif pour Hyperopt
def objective_et(params):
    # Initialiser ExtraTreesClassifier avec les paramètres testés
    model = ExtraTreesClassifier(**params, random_state=42)
    # Utiliser la validation croisée pour évaluer la précision
    score = cross_val_score(model, X_train_hill, y_train, cv=7, scoring='accuracy').mean()
    # Retourner la perte (score négatif) car Hyperopt minimise cette fonction
    return {'loss': -score, 'status': STATUS_OK}

# Lancer l'optimisation avec Hyperopt pour ExtraTreesClassifier
trials_et = Trials()
best_params_et = fmin(
    fn=objective_et,               # Fonction objectif
    space=space_et,                # Espace de recherche
    algo=tpe.suggest,              # Algorithme bayésien pour l'optimisation
    max_evals=50,                  # Nombre d'itérations de recherche
    trials=trials_et,              # Historique des essais
    rstate=np.random.default_rng(42)  # Fixer un seed pour la reproductibilité
)

# Afficher les meilleurs paramètres trouvés
print("Meilleurs paramètres ExtraTreesClassifier :", best_params_et)

100%|██████████| 50/50 [15:32<00:00, 18.65s/trial, best loss: -0.881708238851096] 
Meilleurs paramètres ExtraTreesClassifier : {'max_depth': 0, 'max_features': 2, 'min_samples_leaf': 0, 'min_samples_split': 0, 'n_estimators': 2}


In [21]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report
# Meilleurs paramètres de ExtraTreesClassifier
best_et_params = {
    'n_estimators': [100, 200, 300, 400, 500][2],  
    'max_depth': [None, 10, 20, 30, 40][0],           
    'min_samples_split': [2, 5, 10][0],       
    'min_samples_leaf': [1, 2, 4][0],          
    'max_features': ['sqrt', 'log2', None][2]
}

# Initialiser le modèle LightGBM avec les paramètres optimisés
et_model = ExtraTreesClassifier(**best_et_params, random_state=42, n_jobs=-1)

# Entraîner le modèle sur l'ensemble d'entraînement
et_model.fit(X_train_hill, y_train)

# Prédire les classes sur l'ensemble de test
y_pred_et = et_model.predict(X_test_hill)

# Évaluer les performances du modèle
accuracy_et = accuracy_score(y_test, y_pred_et)
print("Précision du modèle ExtraTrees :", accuracy_et)
print("Rapport de classification ExtraTrees :\n", classification_report(y_test, y_pred_et))

Précision du modèle ExtraTrees : 0.8835978835978836
Rapport de classification ExtraTrees :
               precision    recall  f1-score   support

           1       0.79      0.79      0.79       648
           2       0.80      0.72      0.76       648
           3       0.88      0.90      0.89       648
           4       0.95      0.98      0.97       648
           5       0.92      0.95      0.94       648
           6       0.89      0.89      0.89       648
           7       0.94      0.95      0.94       648

    accuracy                           0.88      4536
   macro avg       0.88      0.88      0.88      4536
weighted avg       0.88      0.88      0.88      4536



## Gradient Boosting

In [22]:
from sklearn.ensemble import GradientBoostingClassifier

# Définir l'espace de recherche pour GradientBoostingClassifier
space_gb = {
    'n_estimators': hp.choice('n_estimators', [ 100, 200, 300, 400]),     # Nombre d'arbres
    'max_depth': hp.choice('max_depth', [5, 7, 10, 12]),                  # Profondeur maximale
    'learning_rate': hp.uniform('learning_rate', 0.05, 0.3),              # Taux d'apprentissage
    'min_samples_split': hp.choice('min_samples_split', [2, 5, 10]),      # Nombre minimum d'échantillons pour diviser
    'min_samples_leaf': hp.choice('min_samples_leaf', [1, 2, 4]),         # Nombre minimum d'échantillons dans une feuille
    'subsample': hp.uniform('subsample', 0.5, 1.0)                        # Fraction des échantillons pour chaque arbre
}

# Fonction objectif pour Hyperopt
def objective_gb(params):
    # Initialiser GradientBoostingClassifier avec les paramètres proposés
    model = GradientBoostingClassifier(**params, random_state=42)
    # Utiliser une validation croisée pour calculer la précision moyenne
    score = cross_val_score(model, X_train_hill, y_train, cv=7, scoring='accuracy').mean()
    # Retourner la perte (score négatif) car Hyperopt minimise cette fonction
    return {'loss': -score, 'status': STATUS_OK}

# Lancer l'optimisation avec Hyperopt pour GradientBoostingClassifier
trials_gb = Trials()
best_params_gb = fmin(
    fn=objective_gb,               # Fonction objectif
    space=space_gb,                # Espace de recherche
    algo=tpe.suggest,              # Algorithme bayésien pour l'optimisation
    max_evals=50,                  # Nombre d'itérations de recherche
    trials=trials_gb,              # Historique des essais
    rstate=np.random.default_rng(42)  # Fixer un seed pour la reproductibilité
)

# Afficher les meilleurs paramètres trouvés
print("Meilleurs paramètres GradientBoostingClassifier :", best_params_gb)

100%|██████████| 50/50 [9:10:03<00:00, 660.07s/trial, best loss: -0.8909674981103554]    
Meilleurs paramètres GradientBoostingClassifier : {'learning_rate': 0.11317932139355968, 'max_depth': 3, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 3, 'subsample': 0.8011279046777869}


In [8]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report

# Meilleurs paramètres de GradientBoostingClassifier
best_gb_params = {
    'n_estimators': 300,  
    'max_depth': 16,#[5, 7, 10, 12][3],           
    'learning_rate': 0.11782851482424293,  
    'min_samples_split': 5,       
    'min_samples_leaf': 6,         
    'subsample': 0.7409699669018692  
}

# Initialiser le modèle GradientBoostingClassifier avec les paramètres optimisés
gb_model = GradientBoostingClassifier(**best_gb_params, random_state=42)

# Entraîner le modèle sur l'ensemble d'entraînement
gb_model.fit(X_train_hill, y_train)

# Prédire les classes sur l'ensemble de test
y_pred_gb = gb_model.predict(X_test_hill)

# Évaluer les performances du modèle
accuracy_gb = accuracy_score(y_test, y_pred_gb)
print("Précision du modèle GradientBoosting :", accuracy_gb)
print("Rapport de classification GradientBoosting :\n", classification_report(y_test, y_pred_gb))

Précision du modèle GradientBoosting : 0.8932980599647267
Rapport de classification GradientBoosting :
               precision    recall  f1-score   support

           1       0.81      0.81      0.81       648
           2       0.82      0.74      0.78       648
           3       0.88      0.90      0.89       648
           4       0.96      0.98      0.97       648
           5       0.93      0.96      0.94       648
           6       0.90      0.91      0.90       648
           7       0.94      0.95      0.95       648

    accuracy                           0.89      4536
   macro avg       0.89      0.89      0.89      4536
weighted avg       0.89      0.89      0.89      4536



In [5]:
import optuna
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

# Fonction objectif pour Optuna
def objective(trial):
    params = {
        'n_estimators': trial.suggest_categorical('n_estimators', [ 300, 400, 450, 500]),
        'max_depth': trial.suggest_categorical('max_depth', [ 10, 12, 14, 16]),
        'learning_rate': trial.suggest_float('learning_rate', 0.05, 0.15),
        'min_samples_split': trial.suggest_categorical('min_samples_split', [ 5, 10, 13]),
        'min_samples_leaf': trial.suggest_categorical('min_samples_leaf', [ 2, 4, 6]),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'random_state': 42
    }

    model = GradientBoostingClassifier(**params)
    score = cross_val_score(model, X_train_hill, y_train, cv=7, scoring='accuracy').mean()
    return score

# Lancer l'optimisation avec Optuna
study = optuna.create_study(direction="maximize", study_name="gb_optimization", storage="sqlite:///optuna_study.db", load_if_exists=True)
study.optimize(objective, n_trials=50, n_jobs=-1)  # Parallélisation avec n_jobs=-1

# Afficher les meilleurs paramètres
print("Best score:", study.best_value)
print("Meilleurs paramètres trouvés :", study.best_params)

[I 2024-12-07 12:00:46,821] Using an existing study with name 'gb_optimization' instead of creating a new one.
[I 2024-12-07 12:20:36,350] Trial 9 finished with value: 0.8881330309901739 and parameters: {'n_estimators': 400, 'max_depth': 10, 'learning_rate': 0.0988326219059468, 'min_samples_split': 13, 'min_samples_leaf': 4, 'subsample': 0.5700002735264409}. Best is trial 9 with value: 0.8881330309901739.
[I 2024-12-07 12:21:57,753] Trial 15 finished with value: 0.8897392290249434 and parameters: {'n_estimators': 500, 'max_depth': 14, 'learning_rate': 0.13511644147178914, 'min_samples_split': 5, 'min_samples_leaf': 6, 'subsample': 0.7894424176815351}. Best is trial 15 with value: 0.8897392290249434.
[I 2024-12-07 12:22:35,429] Trial 11 finished with value: 0.888416477702192 and parameters: {'n_estimators': 450, 'max_depth': 14, 'learning_rate': 0.1339486825543792, 'min_samples_split': 5, 'min_samples_leaf': 4, 'subsample': 0.8689900170747198}. Best is trial 15 with value: 0.88973922902

Best score: 0.8921012849584279
Meilleurs paramètres trouvés : {'n_estimators': 300, 'max_depth': 16, 'learning_rate': 0.11782851482424293, 'min_samples_split': 5, 'min_samples_leaf': 6, 'subsample': 0.7409699669018692}


## Soumission

In [52]:
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report

# Définir les modèles de base avec les meilleurs hyperparamètres
random_forest = RandomForestClassifier(
    n_estimators=300,  
    max_depth=None,      
    random_state=42,
    max_features='sqrt',
    min_samples_split=2,
    min_samples_leaf=1,
)

xgboost = XGBClassifier(
    colsample_bytree=0.9935889419671114,
    gamma=0.012860005064671678,
    learning_rate=0.11209385123371478,
    max_depth=9,
    n_estimators=200,
    subsample=0.7078887451962781,
    objective='multi:softmax',
    num_class=7,
    random_state=42
)

lightgbm = LGBMClassifier(
    colsample_bytree=0.6284626649933062,
    learning_rate=0.08427513495693706,
    max_depth=15,
    n_estimators=200,
    num_leaves=100,
    subsample=0.6894988308344379,
    objective='multiclass',
    num_class=7,
    random_state=42,
    verbosity=-1
)

extra_trees = ExtraTreesClassifier(
    n_estimators=300,
    max_depth=None,
    random_state=42,
    max_features=None,
    min_samples_split=2,
    min_samples_leaf=1,
    n_jobs=-1,
)

gradient_boosting = GradientBoostingClassifier(
    n_estimators=300,
    learning_rate=0.11782851482424293,
    max_depth=16,
    random_state=42,
    min_samples_split=5,
    min_samples_leaf=6, 
    subsample=0.7409699669018692 
)

# Définir le méta-modèle (un modèle simple comme la régression logistique)
meta_model = XGBClassifier(
    n_estimators=160, #150 #200,
    max_depth= 2,  #3,#4
    learning_rate= 0.1,  #0.1,
    colsample_bytree=0.92,
    subsample=0.75,
    random_state=42,
    use_label_encoder=False,
    eval_metric='mlogloss'
)
    # LGBMClassifier(n_estimators=200, learning_rate=0.1, max_depth=5, random_state=42)
    # RandomForestClassifier(n_estimators=200, max_depth=5, random_state=42)
    # LogisticRegression(max_iter=1000)

# Créer le StackingClassifier
stacking_model = StackingClassifier(
    estimators=[
        ('rf', random_forest),
        ('xgb', xgboost),
        ('lgbm', lightgbm),
        ('et', extra_trees),
        ('gb', gradient_boosting)
    ],
    final_estimator=meta_model,
    cv=10  # Utiliser une validation croisée pour le stacking
)

In [16]:
# Entraîner le modèle de stacking
stacking_model.fit(X_train_hill, y_train_adj)

# Prédire sur l'ensemble de test
y_pred_stack = stacking_model.predict(X_test_hill)

# Évaluer les performances du modèle de stacking
accuracy_stack = accuracy_score(y_test_adj, y_pred_stack)
print("Précision du modèle de stacking :", accuracy_stack)
print("Rapport de classification du modèle de stacking :\n", classification_report(y_test_adj, y_pred_stack))

/Users/steph/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [17:00:14] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Précision du modèle de stacking : 0.8996913580246914
Rapport de classification du modèle de stacking :
               precision    recall  f1-score   support

           0       0.82      0.80      0.81       648
           1       0.80      0.80      0.80       648
           2       0.90      0.91      0.90       648
           3       0.97      0.97      0.97       648
           4       0.95      0.94      0.95       648
           5       0.91      0.93      0.92       648
           6       0.96      0.94      0.95       648

    accuracy                           0.90      4536
   macro avg       0.90      0.90      0.90      4536
weighted avg       0.90      0.90      0.90      4536



In [10]:
y_adj = y - 1

X_test_full = pd.read_csv('test-full.csv')

In [34]:
X = X.drop(columns=['Hillshade_9am'])
X_test_full = X_test_full.drop(columns=['Hillshade_9am'])

KeyError: "['Hillshade_9am'] not found in axis"

In [50]:
# On entraîne le modèle de stacking
stacking_model.fit(X, y_adj)

# On prédit sur l'ensemble de test
y_pred_stack = stacking_model.predict(X_test_full)

y_pred_stack = y_pred_stack + 1

/Users/steph/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:08:46] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [51]:
# On crée un DataFrame avec les Id et les Cover_Type prédits
submission_df = pd.DataFrame({
    'Id': X_test_full['Id'],
    'Cover_Type': y_pred_stack
})

# On sauvegarde le DataFrame en fichier CSV
submission_df.to_csv('soumissions/submission_stacking_Hillshade_opt_170.csv', index=False)